# w261 Final Project - Clickthrough Rate Prediction


[Your team number (from the spreadsheet)]   
[Your team names]   
Summer 2019, section [Your section numbers>]   

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

#### LogisticRegression

#### RandomForest

#### GradientBoosting

#### Multilayer Perception

# __Section 3__ - EDA & Challenges

### Section 3.1 - Libraries and Util Functions

#### Section 3.1.1 - Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import time

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import when  
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.sql.functions import lit
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import monotonically_increasing_id

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

#### Section 3.1.2 - Create Spark Context

In [5]:
# start Spark Session
sc_conf = SparkConf()
sc_conf.setAppName("w261_final_project")

spark = SparkSession\
        .builder\
        .config(conf=sc_conf)\
        .getOrCreate()
    
sc = spark.sparkContext

#### Section 3.1.3 - Util Functions

In [ ]:
#Fill missing values - numerical
def fill_missing_val(t_data):    
    return t_data.na.fill(0)

#Get numeric and categorical columns
def get_columns():
    num_cols = []
    cat_cols = []
    
    #Numerical columns range from column index 2-13
    for c in range(2, 14):
        num_cols.append("_" + str(c))
    
    #Categorical columns range from column index 14-40
    for c in range(14, 41):
        cat_cols.append("_" + str(c))
    
    return num_cols, cat_cols

In [10]:
#Load the data

df = spark.read.csv('data/dac/train.small.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- _1: integer (nullable = true)
 |-- _2: integer (nullable = true)
 |-- _3: integer (nullable = true)
 |-- _4: integer (nullable = true)
 |-- _5: integer (nullable = true)
 |-- _6: integer (nullable = true)
 |-- _7: integer (nullable = true)
 |-- _8: integer (nullable = true)
 |-- _9: integer (nullable = true)
 |-- _10: integer (nullable = true)
 |-- _11: integer (nullable = true)
 |-- _12: integer (nullable = true)
 |-- _13: integer (nullable = true)
 |-- _14: integer (nullable = true)
 |-- _15: string (nullable = true)
 |-- _16: string (nullable = true)
 |-- _17: string (nullable = true)
 |-- _18: string (nullable = true)
 |-- _19: string (nullable = true)
 |-- _20: string (nullable = true)
 |-- _21: string (nullable = true)
 |-- _22: string (nullable = true)
 |-- _23: string (nullable = true)
 |-- _24: string (nullable = true)
 |-- _25: string (nullable = true)
 |-- _26: string (nullable = true)
 |-- _27: string (nullable = true)
 |-- _28: string (nullable = true)
 |-- _29: 

In [11]:
#Store the original columns
cols = df.columns

In [12]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
_1,0,0,1,0,0
_2,0,0,NaN,NaN,4
_3,0,8,1,-1,64
_4,2,NaN,3,NaN,4
_5,9,NaN,20,NaN,2
_6,1366,4910,17647,311637,1490
_7,316,NaN,30,NaN,50
_8,24,NaN,1,0,6
_9,47,8,1,0,12
_10,350,NaN,30,1,27


In [16]:
#Get numeric and categorical column names
numericCols = df.columns[1:14]
categoricalColumns = df.columns[16:41]

print(numericCols)
print(categoricalColumns)

['_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14']
['_17', '_18', '_19', '_20', '_21', '_22', '_23', '_24', '_25', '_26', '_27', '_28', '_29', '_30', '_31', '_32', '_33', '_34', '_35', '_36', '_37', '_38', '_39', '_40']


In [17]:
#Data distribution

df.describe().select('Summary', '_2', '_3', '_4', '_5', '_6').show()

+-------+------------------+----------------+-----------------+-----------------+-----------------+
|Summary|                _2|              _3|               _4|               _5|               _6|
+-------+------------------+----------------+-----------------+-----------------+-----------------+
|  count|              2764|            5000|             4014|             4012|             4859|
|   mean|3.0861070911722144|        111.8884|20.57498754359741|7.587487537387837|19752.57089936201|
| stddev| 7.437360728411959|398.904083256824|58.24237003906267|8.949486932145078|74237.14669289338|
|    min|                 0|              -2|                0|                0|                0|
|    max|               151|            5945|             2050|               91|          1577121|
+-------+------------------+----------------+-----------------+-----------------+-----------------+



In [18]:
df.describe().select('Summary', '_7', '_8', '_9', '_10', '_11').show()

+-------+------------------+------------------+------------------+------------------+------------------+
|Summary|                _7|                _8|                _9|               _10|               _11|
+-------+------------------+------------------+------------------+------------------+------------------+
|  count|              3800|              4778|              4999|              4778|              2764|
|   mean|124.05131578947369|15.510464629552114|12.320464092818563|103.23231477605692| 0.532561505065123|
| stddev| 347.0117403373191|48.293716133203866|13.536432961145458|210.93376545285946|0.6234846166130683|
|    min|                 0|                 0|                 0|                 0|                 0|
|    max|              5936|              1036|               237|              4144|                 5|
+-------+------------------+------------------+------------------+------------------+------------------+



In [19]:
df.describe().select('Summary', '_12', '_13', '_14').show()

+-------+-----------------+------------------+------------------+
|Summary|              _12|               _13|               _14|
+-------+-----------------+------------------+------------------+
|  count|             4778|              1152|              4012|
|   mean|2.620971117622436|0.8732638888888888| 8.593469591226322|
| stddev|5.097635360270372|3.5293847192911856|11.277355438010085|
|    min|                0|                 0|                 0|
|    max|               81|                43|               182|
+-------+-----------------+------------------+------------------+



In [20]:
#Fill missing values - numerical

df = df.na.fill(0)

#for col in numericCols:
#    df= df.withColumn(col, when(df[col]>0,1).otherwise(0))


#imputer = Imputer(inputCols = numericCols, outputCols = numericCols)
#imp_model = imputer.fit(df)

#df = imp_model.transform(df)
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
_1,0,0,1,0,0
_2,0,0,0,0,4
_3,0,8,1,-1,64
_4,2,0,3,0,4
_5,9,0,20,0,2
_6,1366,4910,17647,311637,1490
_7,316,0,30,0,50
_8,24,0,1,0,6
_9,47,8,1,0,12
_10,350,0,30,1,27


In [21]:
print(df[df['_1'] == 0].count())
print(df[df['_1'] == 1].count())

3729
1271


In [58]:
#df.fillna(df.mean())

#df['_c1'].fillna((df['_c1'].mean()), inplace=True)

In [22]:
stages = []
indexerCols = []

for categoricalCol in categoricalColumns:
    indexerCol = categoricalCol + "Index"
    indexer = StringIndexer(inputCol=categoricalCol, outputCol= indexerCol).setHandleInvalid("keep")
    stages += [indexer]
    indexerCols.append(indexerCol)

label_stringIdx = StringIndexer(inputCol = '_1', outputCol = 'output')
stages += [label_stringIdx]

In [23]:
assembler = VectorAssembler(inputCols=indexerCols + numericCols, outputCol="features")
stages += [assembler]

In [24]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)

selectedCols = ['output', 'features'] + cols
df = df.select(selectedCols)

pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
output,0,0,1,0,0
features,"[156.0, 220.0, 0.0, 0.0, 31.0, 4.0, 0.0, 1.0, ...","(1316.0, 1446.0, 0.0, 5.0, 462.0, 3.0, 0.0, 77...","[1452.0, 86.0, 0.0, 1.0, 970.0, 0.0, 0.0, 18.0...","[598.0, 2133.0, 0.0, 1.0, 174.0, 1.0, 1.0, 560...","[58.0, 69.0, 0.0, 1.0, 313.0, 2.0, 0.0, 1176.0..."
_1,0,0,1,0,0
_2,0,0,0,0,4
_3,0,8,1,-1,64
_4,2,0,3,0,4
_5,9,0,20,0,2
_6,1366,4910,17647,311637,1490
_7,316,0,30,0,50
_8,24,0,1,0,6


In [25]:
standardscaler=StandardScaler().setInputCol("features").setOutputCol("scaled_features")
df=standardscaler.fit(df).transform(df)
df.select("features","Scaled_features").show(5)

+--------------------+--------------------+
|            features|     Scaled_features|
+--------------------+--------------------+
|[156.0,220.0,0.0,...|[0.14742378229343...|
|(37,[0,1,3,4,5,7,...|(37,[0,1,3,4,5,7,...|
|[1452.0,86.0,0.0,...|[1.37217520442350...|
|[598.0,2133.0,0.0...|[0.56512449879149...|
|[58.0,69.0,0.0,1....|[0.05481140623730...|
+--------------------+--------------------+
only showing top 5 rows



In [26]:
#Train test split

train, test = df.randomSplit([0.7, 0.3], seed = 2019)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 3476
Test Dataset Count: 1524


In [27]:
#Imbalance check

dataset_size=float(train.select("output").count())
numPositives=train.select("output").where('output == 1').count()
per_ones=(float(numPositives)/float(dataset_size))*100
numNegatives=float(dataset_size-numPositives)
print('The number of ones are {}'.format(numPositives))
print('Percentage of ones are {}'.format(per_ones))

The number of ones are 897
Percentage of ones are 25.805523590333717


In [28]:
#Rebalance data

BalancingRatio= numNegatives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))

BalancingRatio = 0.7419447640966629


In [29]:
train=train.withColumn("classWeights", when(train.output == 1,BalancingRatio).otherwise(1-BalancingRatio))
train.select("classWeights").show(5)

+------------------+
|      classWeights|
+------------------+
|0.2580552359033371|
|0.2580552359033371|
|0.2580552359033371|
|0.2580552359033371|
|0.2580552359033371|
+------------------+
only showing top 5 rows



In [30]:
#Feature selection

css = ChiSqSelector(featuresCol='scaled_features',outputCol='Aspect',labelCol='output',fpr=0.05)
train=css.fit(train).transform(train)
test=css.fit(test).transform(test)
test.select("Aspect").show(5,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Aspect                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

### Logistic Regression

In [ ]:
lr = LogisticRegression(maxIter=10, featuresCol="Aspect", labelCol="output", 
                        weightCol="classWeights", predictionCol="prediction")

# Fit the model
lrModel = lr.fit(train)

predict_train=lrModel.transform(train)
predict_test=lrModel.transform(test)

In [ ]:
#Evaluate the model

beta = np.sort(lrModel.coefficients)
plt.plot(beta)
plt.ylabel('Beta Coefficients')
plt.show()

In [ ]:
trainingSummary = lrModel.summary
roc = trainingSummary.roc.toPandas()
plt.plot(roc['FPR'],roc['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

In [ ]:
pr = trainingSummary.pr.toPandas()
plt.plot(pr['recall'],pr['precision'])
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.show()

In [ ]:
predict_test.select("output","prediction").show(10)

In [ ]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.aggregationDepth,[2,10])\
    .addGrid(lr.elasticNetParam,[0.0, 0.5])\
    .addGrid(lr.maxIter,[10, 100])\
    .addGrid(lr.regParam,[0.01, 0.5, 2.0]) \
    .build()

In [38]:
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=2)
# Run cross validations
cvModel = cv.fit(train)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_train=cvModel.transform(train)
predict_test=cvModel.transform(test)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_train)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(predict_test)))


The area under ROC for train set after CV  is 0.7101656765496804
The area under ROC for test set after CV  is 0.6967054173448046


#### RF

In [ ]:
rf = RandomForestClassifier(numTrees=10, featuresCol="scaled_features", labelCol="output", predictionCol="prediction")
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('output', 'prediction', 'probability').show(10)

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'rawPrediction', labelCol = 'output')

predict_test.select("output","rawPrediction","prediction","probability").show(5)
print("The area under ROC for train set is {}".format(evaluator.evaluate(predict_train)))
print("The area under ROC for test set is {}".format(evaluator.evaluate(predict_test)))

#### GB

In [ ]:
gbt = GBTClassifier(maxIter=10, featuresCol="scaled_features", labelCol="output", predictionCol="prediction")
gbtModel = gbt.fit(train)
predictions = gbtModel.transform(test)
predictions.select('output', 'prediction', 'probability').show(10)

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol = 'output')
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

In [ ]:
print(gbt.explainParams())

# __Section 4__ - Algorithm Implementation (Cloud Processing with Full Data)

### Section 4.1 - File and Data Pre-processing

In [ ]:
BUCKET_BASE_PATH = 'gs://w261FinalProj/projdata/'
DATABRICKS_NS = "com.databricks.spark.csv"

#### Section 4.1.1 - Generate Dataframe from Input Text data

In [ ]:
#Read input data as an RDD
train_tmp_rdd = sc.textFile(BUCKET_BASE_PATH + 'dac/train.txt')

#Format the tab delimited item before converting to dataframe
rdd_train = train_tmp_rdd.map(lambda r : r.split('\t'))

#Convert the RDD to a dataframe
df_train = rdd_train.toDF()

#Saves 
df_train.write.format(DATABRICKS_NS).option("header", "true").save(BUCKET_BASE_PATH + "dac/train.csv")

#### Section 4.1.2 - Split Input Dataframe by Columns

In [ ]:
#Read csv as a dataframe
df_train = spark.read.csv(BUCKET_BASE_PATH + 'dac/train.csv', header = True, inferSchema = True)

#Get all the columns of the dataframe
cols = df_train.columns

#Iterate columns and generate dataframes for each
for col in cols:
    df_with_one_col = df_train.select(col)
    df_with_one_col.write.format(DATABRICKS_NS).option("header", "true").save(BUCKET_BASE_PATH + "split_df/train" + col + ".csv")

#### Section 4.1.3 - Generate StringIndexer for each Categorical Column

In [ ]:
#Convert categorical string values to ordered numeric frequencies
def get_cat_col_indexed(t_data, index_col):
    #Keep the unseen label and create a new frequency index
    indexer = StringIndexer(inputCol=t_data.columns[0], outputCol= index_col).setHandleInvalid("keep")
    
    t_data = indexer.fit(t_data).transform(t_data)
    
    #Return a dataframe with only indexed column
    t_data = t_data.select(index_col)
    
    return t_data


def pre_process_each_split_df(t_data, index_col):    
    #Fill missing values
    t_data = fill_missing_val(t_data)
    
    return get_cat_col_indexed(t_data, index_col)

#Convert each categorical string values to numerical frequencies with StringIndexer
def indexed_all_dfs(cols):
    #Read each dataframe by categorical columns and save each of them after processing
    for c in cols:
        df_train = spark.read.csv(BUCKET_BASE_PATH + "split_df/train" + c + ".csv", header = True, inferSchema = True)
        index_col = df_train.columns[0] + "Index"
        df_train = pre_process_each_split_df(df_train, index_col)
        df_train.write.format(DATABRICKS_NS).option("header", "true").save(BUCKET_BASE_PATH + "split_df_indx/train_p_" + c + ".csv")
        
    #Process the output column
    df_train = spark.read.csv(BUCKET_BASE_PATH + "split_df/train_1.csv", header = True, inferSchema = True)
    df_train = get_cat_col_indexed(df_train, "output")
    df_train.write.format(DATABRICKS_NS).option("header", "true").save(BUCKET_BASE_PATH + "split_df_indx/train_p_output.csv")

In [ ]:
#Get columns and stages
num_cols, cat_cols = get_columns()

indexed_all_dfs(cat_cols)

#### Section 4.1.4 - Generate Vector Assembler Column

In [ ]:
def print_to_output_file(p_txt):
    print(p_txt)

#Get all the numerical and categorical columns and create a vector for the ML algos
def get_vect_assem(t_data, cols):    
    assembler = VectorAssembler(inputCols=cols, outputCol="features")

    return assembler.transform(t_data)

#Read each indexed (categorical) / numeric columns and generate a vector column
def assemble_all_dfs():    
    num_cols, cat_cols = get_columns()
    cat_indexed_col = []
    
    #Read and join each dataframes with auto-generated id columns
    df_train = spark.read.csv(BUCKET_BASE_PATH + "split_df_indx/train_p_output.csv", header = True, inferSchema = True)
    df_train = df_train.withColumn("id", monotonically_increasing_id())
    
    for nc in num_cols:
        df1 = spark.read.csv(BUCKET_BASE_PATH + "split_df/train" + nc + ".csv", header = True, inferSchema = True)
        df1 = df1.withColumn("id", monotonically_increasing_id())
        df_train = df_train.join(df1, "id", "outer")
        
    for c in cat_cols:
        df1 = spark.read.csv(BUCKET_BASE_PATH + "split_df_indx/train_p_" + c + ".csv", header = True, inferSchema = True)
        df1 = df1.withColumn("id", monotonically_increasing_id())
        df_train = df_train.join(df1, "id", "outer")
        cat_indexed_col.append(df1.columns[0])
    
    #After joining all the dataframes, remove the id column 
    df_train = df_train.drop('id')
    
    all_c = num_cols + cat_indexed_col
        
    df_train = fill_missing_val(df_train)
    
    df_train = get_vect_assem(df_train, all_c).select(['features', 'output'])
        
    return df_train  


In [ ]:
t0 = time.time()

df_tn = assemble_all_dfs()
t1 = time.time()
print_to_output_file('Runtime - assemble all - ' + str(float(t1 - t0)))

Runtime - assemble all - 297.4144537448883

#### Section - 4.1.5 - Standardize Scales for all the Values

In [ ]:
def run_standard_scaler(t_data):
    standardscaler=StandardScaler().setInputCol("features").setOutputCol("scaled_features")
    t_data = standardscaler.fit(t_data).transform(t_data)
    
    return t_data

In [ ]:
t0 = time.time()

df_tn = run_standard_scaler(df_tn)

t1 = time.time()
print_to_output_file('Runtime - standard scaler - ' + str(float(t1 - t0)))

Runtime - standard scaler - 1022.7532405853271

#### Section - 4.1.6 - Test-Train split

In [ ]:
def train_test_split(t_data):
    train, test = t_data.randomSplit([0.7, 0.3], seed = 2019)
    print("Training Dataset Count: " + str(train.count()))
    print("Test Dataset Count: " + str(test.count()))
    
    return train, test

In [ ]:
t0 = time.time()

train, test = train_test_split(df_tn)

t1 = time.time()
print_to_output_file('Runtime - test train split - ' + str(float(t1 - t0)))

Training Dataset Count: 32195163 <br>
Test Dataset Count: 13799653 <br>
Runtime - test train split - 2079.2427320480347

#### Section - 4.1.7 - Balance Train Data Weight Ratio

In [ ]:
def get_balance_weight_ratio_data(t_data):
    dataset_size=float(t_data.select("output").count())
    numPositives=t_data.select("output").where('output == 1').count()
    per_ones=(float(numPositives)/float(dataset_size))*100
    numNegatives=float(dataset_size-numPositives)
    
    #Rebalance data
    BalancingRatio = numNegatives/dataset_size
    
    return t_data.withColumn("classWeights", when(t_data.output == 1,BalancingRatio).otherwise(1-BalancingRatio))

In [ ]:
t0 = time.time()

train = get_balance_weight_ratio_data(train)

t1 = time.time()
print_to_output_file('Runtime - get balanced train data - ' + str(float(t1 - t0)))

Runtime - get balanced train data - 2556.4728071689606

#### Section - 4.1.8 - Print Performance

In [ ]:
def print_perf_summary(trainingSummary):
    print_to_output_file('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))
    
    accuracy = trainingSummary.accuracy
    falsePositiveRate = trainingSummary.weightedFalsePositiveRate
    truePositiveRate = trainingSummary.weightedTruePositiveRate
    fMeasure = trainingSummary.weightedFMeasure()
    precision = trainingSummary.weightedPrecision
    recall = trainingSummary.weightedRecall
    
    trainingOutput = "Accuracy: " + str(accuracy) + "\nFPR: " + str(falsePositiveRate) + "\nTPR: " + str(truePositiveRate) + "\nF-measure: " + str(fMeasure) + "\nPrecision: " + str(precision) + "\nRecall: " + str(recall)
    
    print_to_output_file(trainingOutput)
    
def print_perf_eval(predictions):
    predictions.filter(predictions['prediction'] == 0) \
    .select("probability","output","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)
    
    evaluator = BinaryClassificationEvaluator(labelCol = 'output')
    print_to_output_file("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

### Logistic Regression

In [31]:
lr = LogisticRegression(maxIter=10, featuresCol="scaled_features", labelCol="output", 
                            weightCol="classWeights", predictionCol="prediction")
    # Fit the model
    lrModel = lr.fit(tn_data)

    predict_train=lrModel.transform(tn_data)
    #predict_test=lrModel.transform(ts_data)
    
    trainingSummary = lrModel.summary
    
    print_perf_summary(trainingSummary)

In [ ]:
t0 = time.time()

run_logistic_regression(train)

t1 = time.time()
print_to_output_file('Runtime - lr - ' + str(float(t1 - t0)))

Training set areaUnderROC: 0.5840384388912582 <br>
Accuracy: 0.4806336591617815 <br>
FPR: 0.36701534264209335 <br>
TPR: 0.4806336591617815 <br>
F-measure: 0.5035350320477693 <br>
Precision: 0.671720105676888 <br>
Recall: 0.4806336591617815 <br>
Runtime - lr - 3954.8369500637054 <br>



### RF

In [ ]:
def run_random_forest_algorithm(tn_data, ts_data):
    rf = RandomForestClassifier(numTrees=10, featuresCol="scaled_features", labelCol="output", predictionCol="prediction")
    rfModel = rf.fit(tn_data)
    predictions = rfModel.transform(ts_data)
    
    print_perf_eval(predictions)

In [ ]:
t0 = time.time()

run_random_forest_algorithm(train, test)

t1 = time.time()
print_to_output_file('Runtime - rf' + str(float(t1 - t0)))

Test Area Under ROC: 0.617709968737223 <br>
Runtime - rf - 4965.246869325638

### GB

In [ ]:
def run_gradient_boost(tn_data, ts_data):
    gbt = GBTClassifier(maxIter=10, featuresCol="scaled_features", labelCol="output", predictionCol="prediction")
    gbtModel = gbt.fit(tn_data)
    predictions = gbtModel.transform(ts_data)
    
    print_perf_eval(predictions)

In [ ]:
t0 = time.time()

run_gradient_boost(train, test)

t1 = time.time()
print_to_output_file('Runtime - gb' + str(float(t1 - t0)))

Test Area Under ROC: 0.6455974577301661 <br>
Runtime - gb5203.547702550888

#### LSVC

In [ ]:
def run_lsvc(tn_data, ts_data):
    sv = LinearSVC(maxIter=10, regParam=0.1,
                     featuresCol="scaled_features", labelCol="output", predictionCol="prediction")
    svModel = sv.fit(tn_data)
    predictions = svModel.transform(ts_data)
    
    evaluator = BinaryClassificationEvaluator(labelCol = 'output')
    print_to_output_file("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

In [ ]:
t0 = time.time()

run_lsvc(train, test)

t1 = time.time()
print_to_output_file('Runtime - lsvc - ' + str(float(t1 - t0)))


19/08/03 10:34:50 INFO breeze.optimize.OWLQN: Step Size: 0.1392  
19/08/03 10:34:50 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.670068 (rel: 0.330) 1.04394  
19/08/03 10:36:58 INFO breeze.optimize.OWLQN: Step Size: 0.5000  
19/08/03 10:36:58 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.648529 (rel: 0.0321) 0.578895  
19/08/03 10:39:07 INFO breeze.optimize.OWLQN: Step Size: 0.5000  
19/08/03 10:39:07 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.629761 (rel: 0.0289) 0.469928  
19/08/03 10:41:16 INFO breeze.optimize.OWLQN: Step Size: 0.5000  
19/08/03 10:41:16 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.605813 (rel: 0.0380) 0.135694  
19/08/03 10:43:24 INFO breeze.optimize.OWLQN: Step Size: 0.5000  
19/08/03 10:43:24 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.603570 (rel: 0.00370) 0.129355  
19/08/03 10:45:33 INFO breeze.optimize.OWLQN: Step Size: 0.5000  
19/08/03 10:45:33 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.596887 (rel: 0.0111) 0.162369  
19/08/03 10:47:42 INFO breeze.optimize.OWLQN: Step Size: 0.5000  
19/08/03 10:47:42 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.588989 (rel: 0.0132) 0.235722  
19/08/03 10:49:51 INFO breeze.optimize.OWLQN: Step Size: 0.5000  
19/08/03 10:49:51 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.576374 (rel: 0.0214) 0.204721  
19/08/03 10:52:00 INFO breeze.optimize.OWLQN: Step Size: 0.5000  
19/08/03 10:52:00 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.553033 (rel: 0.0405) 0.379363  
19/08/03 10:56:20 INFO breeze.optimize.OWLQN: Step Size: 0.1250  
19/08/03 10:56:20 INFO breeze.optimize.OWLQN: Val and Grad Norm: 0.547216 (rel: 0.0105) 0.209765  
19/08/03 10:56:20 INFO breeze.optimize.OWLQN: Converged because max iterations reached  
Test Area Under ROC: 0.5563562601488108  
Runtime - lsvc - 4158.405479669571  

# __Section 5__ - Course Concepts